In [3]:
import os
import cv2
import math
import random
import numpy as np
from PIL import Image
import types
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from torch.autograd import Variable
from torchvision import datasets, transforms, models
from sklearn.metrics import classification_report
from torch import optim
import time

In [4]:
# parameters setting
input_shape = [3,96,96]
num_classes = 7
cut_size = 225
init_lr = 0.0003
batch_size = 256
use_fer=False
device= torch.device("cuda")

In [5]:
# load data
data_dir = ''

if use_fer:
    full_data = torch.load(data_dir + 'pickles/fer2013.pt')
    full_img, full_emo = full_data['images'][:,None,:,:], full_data['label']

    train_img,train_y = full_img[:28709],full_emo[:28709]
    test_img,test_y = full_img[28709:28709+3589],full_emo[28709:28709+3589]

    train_img = torch.cat((train_img,train_img,train_img),1)
    test_img = torch.cat((test_img,test_img,test_img),1)

else:
    train_data = torch.load(data_dir + 'pickles/AffectNet_train.pt')
    train_img, train_y = train_data['images'], train_data['emotion']
    
    test_data = torch.load(data_dir + 'pickles/AffectNet_test.pt')
    test_img, test_y = test_data['images'], test_data['emotion']

In [6]:
def nine_crops(image, size=[96,96], n_crop=3):
    xx = yy = n_crop
    x = size[0] // xx
    y = size[1] // yy
    crops = ()
    for j in range(yy):
        for i in range(xx):
            left = i*x
            up = y*j
            right = left + x
            low = up + y
            region = image.crop((left,up,right,low))
            region = transforms.RandomCrop(64)(region)
            region = transforms.Resize((75, 75), Image.BILINEAR)(region)
#             region = transforms.Lambda(rgb_jittering)(region)
            crops += (region,)       
    return crops


# def rgb_jittering(im):
#     im = np.array(im, 'int32')
#     for ch in range(3):
#         im[:, :, ch] += np.random.randint(-2, 2)
#     im[im > 255] = 255
#     im[im < 0] = 0
#     return im.astype('uint8')


class PuzzleCrop(object):
    def __init__(self, size, n_crop):
        self.size = size
        self.n_crop = n_crop

    def __call__(self, img):
        return nine_crops(img, self.size, self.n_crop)


class Lambda(object):
    def __init__(self, lambd):
        assert isinstance(lambd, types.LambdaType)
        self.lambd = lambd
    def __call__(self, img):
        return self.lambd(img)

In [28]:
from torch.utils.data import Dataset, DataLoader
#======= data loaders =======#    
n_crop = 3
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([int(cut_size*1.1),int(cut_size*1.1)]),
    transforms.RandomCrop([cut_size,cut_size]),
    transforms.RandomHorizontalFlip(),
    PuzzleCrop([cut_size,cut_size],3),
    Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops]))
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    PuzzleCrop([cut_size,cut_size],3),
    Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops]))
])

resize_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([cut_size,cut_size]),
    transforms.ToTensor()
])

def default_loader(images):
    img_tensor = train_transform(images)
    return img_tensor

def default_test_loader(images):
    img_tensor = test_transform(images)
    return img_tensor

class Dataset(Dataset):
    def __init__(self,imgs,ys,loader=default_loader):
        self.images = imgs 
        self.label = ys
        self.loader = loader

    def __getitem__(self, index):

        img = self.images[index]
        y = self.label[index]
        n_imgs = self.loader(img)
        img = resize_transform(img)
        return img,n_imgs,y

    
    def __len__(self):
        return len(self.images)

In [8]:
class Puzzle9n1(nn.Module):

    def __init__(self,lr=0.0001,num_classes=9):
        super(Puzzle9n1, self).__init__()
        
        alexnet = models.alexnet(pretrained=True)
        
        self.feature = alexnet.features
        self.classifier = nn.Sequential(
            nn.Linear(9472,4608),
            nn.ReLU(),
            nn.Linear(4608,4096),
            nn.ReLU(),
            nn.Linear(4096,100),
            nn.ReLU(),
            nn.Linear(100, num_classes)
        )
        
        self.loss_fn = nn.CrossEntropyLoss() 
        self.optimizer=torch.optim.Adam([p for p in self.parameters() if p.requires_grad] , lr=lr)
        

    def forward(self,x,n_x):
        bs, n_puzzle, c, h, w = n_x.size() 
        n_x = n_x.view(-1, c, h, w)
        f = self.feature(n_x).view(bs,n_puzzle,-1) # features of each tile
        z = self.feature(x).view(bs,1,-1) # features of full image
        z = z.expand(bs,n_puzzle,-1)
        f_all = torch.cat((f,z),-1)
        return f_all
    
        
    def train_model(self, data_loader, num_epochs, lr, min_loss):
        self.train()
        start=time.time()

        for epoch in range(num_epochs):
            running_loss=0
            running_acc = 0.
            total = 0
            num_batches = 0
            for count, (images, n_images, label) in enumerate(data_loader):
                self.optimizer.zero_grad()
                bs, n_puzzle, c, h, w = n_x.size() 
                images=images.to(device)
                n_images=n_images.to(device)
                label=label.to(device)

                features = self.forward(images, n_images)
                scores = self.classifier(features.view(bs, n_puzzle, -1))
                loss =  self.loss_fn(scores, label) 
                loss.backward()

                self.optimizer.step()

                running_loss += loss.detach().item()

                acc = (scores.argmax(-1) == label).float().sum()
                running_acc += acc.item()/9

                total+=images.size(0)
                num_batches += 1
            total_loss = running_loss/num_batches
            total_acc = running_acc/total
            elapsed = time.time()-start
            print('\t', 'epoch=',epoch, '\t time = {:.0f}m {:.0f}s'.format(elapsed // 60, elapsed % 60),'\t lr=', lr,'\t loss = ', total_loss , '\t Acc = {:.6f}'.format(total_acc*100),'%')
        return total_loss

In [30]:
class transfer(nn.Module):

    def __init__(self,model=None,lr=0.0001,num_classes=9):
        super(transfer, self).__init__()
        self.model = model
        
#         for p in self.model.encoder.parameters():
#                 p.requires_grad = False
        
#         for p in self.model.decoder.parameters():
#                 p.requires_grad = False
        
        self.classifier1 = nn.Sequential(
            nn.Conv2d(256,1024,3,1),
            nn.ReLU(),
            nn.BatchNorm2d(1024),
            nn.Conv2d(1024,1024,3,1),
            nn.ReLU(),
            nn.BatchNorm2d(1024),
        )
        self.classifier2 = nn.Sequential(
            nn.Linear(4096,1024),
            nn.ReLU(),
            nn.Linear(1024,7)
        )
        
        self.loss_fn = nn.CrossEntropyLoss()
        self.optimizer=torch.optim.Adam([p for p in self.parameters() if p.requires_grad] , lr=lr)
    def forward(self, x):
        B,T,C,H,W = x.size()
        x = x.transpose(0,1)
        x_list = []
        for i in range(9):
            z = self.model.feature(x[i])
#             z = self.classifier1(z)
            z = z.view([B,1,-1])
            x_list.append(z)
        x = torch.cat(x_list,1)
        x = x.view(B,256,3,3)
        return x

    def train_model(self, data_loader, num_epochs, lr):
        self.train()
        start=time.time()

        for epoch in range(num_epochs):
            running_loss=0
            running_acc = 0.
            total = 0
            num_batches = 0
            for count, (images, n_images, label) in enumerate(data_loader):
                self.optimizer.zero_grad()

                scores = 0.
                
                images=images.to(device)
                n_images=n_images.to(device)

                feat = self.model.feature(images)
                x = self.classifier1(feat).view(images.size(0),-1)
                scores = self.classifier2(x)

                label=label.to(device)
                loss =  self.loss_fn(scores, label) 
                loss.backward()

                self.optimizer.step()

                running_loss += loss.detach().item()

                acc = (scores.argmax(-1) == label).float().sum()
                running_acc += acc.item()

                total+=images.size(0)
                num_batches += 1
                
            total_loss = running_loss/num_batches
            total_acc = running_acc/total
            elapsed = time.time()-start
            print('\t', 'epoch=',epoch, '\t time = {:.0f}m {:.0f}s'.format(elapsed // 60, elapsed % 60),'\t lr=', lr,'\t loss = ', total_loss , '\t Acc = {:.6f}'.format(total_acc*100),'%')

    def test_model(self, data_loader):
        self.eval()
        start=time.time()
        
        running_acc = 0
        total = 0
        num_batches = 0
        
        preds = []
        truth = []
        
        for count, (images, label) in enumerate(data_loader):
            self.optimizer.zero_grad()
            scores = 0.
            images = images.to(device)
            
            n_images=n_images.to(device)

            feat = self.model.feature(images)
            x = self.classifier1(feat).view(images.size(0),-1)
            scores = self.classifier2(x).detach().cpu()

            acc = (scores.argmax(-1) == label).float().sum()
            running_acc += acc.item()
            
            total += images.size(0)
            num_batches += 1
            
            preds.extend(scores.argmax(-1).numpy())
            truth.extend(label.numpy())
            
        total_acc = running_acc/total
        elapsed = time.time()-start

        print('testing','\t time = {:.0f}m {:.0f}s'.format(elapsed // 60, elapsed % 60), '\n Acc = {:.6f}'.format(total_acc*100),'%')
        test_fscore = classification_report(truth, preds)
        print(test_fscore)

In [ ]:
train_input  = Dataset(train_img,train_y)
train_loader = DataLoader(train_input, batch_size=batch_size,shuffle=False)

test_input  = Dataset(test_img,test_y,default_test_loader)
test_loader = DataLoader(test_input, batch_size=batch_size,shuffle=False)

target_shape=[train_input.images.size(1), 96, 96]

model = Puzzle9n1(lr=init_lr)
model.load_state_dict(torch.load(data_dir + 'baselines/Puzzle_AffectNet_inputsize225_minLoss.pth'))

net = transfer(model=model)
net.to(device)
print(net)
for i in range(10):
    print('Iteration', i)
    net.train_model(data_loader=train_loader, num_epochs=5,lr=init_lr)
    net.test_model(data_loader=test_loader)   